This notebook demonstrates how to use OptKeras, a Python wrapper around Keras and Optuna

##  Set up Google Colab environment

To run in Google Colab, specify a directory in Google Drive. (GPU is recommended.)

To run in an environment other than Google Colab, just skip this code.

In [1]:
import os

try:
    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    # Specify a directory in Google Drive (Modify as in your Google Drive)
    workdir = '/content/drive/My Drive/Colab Notebooks/OptKeras_Example_Output'
    # Create target directory & all intermediate directories if don't exists
    if not os.path.exists(workdir):
        os.makedirs(workdir)
        print('## Directory: ' , workdir ,  ' was created.') 
    os.chdir(workdir)
    print('## Current working directory was set to: ', os.getcwd())
    print('## Check the uptime. (Google Colab reboots every 12 hours)')
    !cat /proc/uptime | awk '{print "Uptime is " $1 /60 /60 " hours (" $1 " sec)"}'
    print('## Check the GPU info')
    !nvidia-smi
    print('## Check the OS') 
    !cat /etc/issue
    print('## Check the Python version') 
    !python --version
    print('## Check the memory')
    !free -h
    print('## Check the disk')
    !df -h
except:
    print('Run the code assuming the environment is not Google Colab.')

 


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
## Current working directory was set to:  /content/drive/My Drive/Colab Notebooks/OptKeras_Example_Output
## Check the uptime. (Google Colab reboots every 12 hours)
Uptime is 0.030125 hours (108.45 sec)
## Check the GPU info
Sun Feb 24 09:31:40 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.79       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+--------------

## Install Optuna 0.7.0

In [2]:
!pip install optuna==0.7.0

    100% |████████████████████████████████| 61kB 2.9MB/s 
    100% |████████████████████████████████| 81kB 7.6MB/s 
    100% |████████████████████████████████| 92kB 29.6MB/s 
    100% |████████████████████████████████| 51kB 23.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/8c/97/c7/868e9cd5d6a23cfb98266f30d59131e5bd441132bdf8da18d5
  Stored in directory: /root/.cache/pip/wheels/92/f0/ac/2ba2972034e98971c3654ece337ac61e546bdeb34ca960dc8c
Successfully built optuna pyperclip


## Install OptKeras 0.0.1

In [3]:
!pip install optkeras==0.0.1
#!pip install git+https://github.com/Minyus/optkeras.git

## Import modules

In [4]:
import numpy as np
import pandas as pd

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Activation, Flatten, Dense, Conv2D
from keras.layers import MaxPooling2D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Adagrad, RMSprop, Adam, Adadelta, Adamax, Nadam
import keras.backend as K

import keras
print('Keras', keras.__version__)

import tensorflow as tf
print('TensorFlow', tf.__version__)

# import Optuna and OptKeras after Keras
import optuna 
print('Optuna', optuna.__version__)

from optkeras.optkeras import OptKeras
import optkeras
print('OptKeras', optkeras.__version__)

# (Optional) Disable messages from Optuna below WARN level.
optuna.logging.set_verbosity(optuna.logging.WARN) 

Using TensorFlow backend.


Keras 2.2.4
TensorFlow 1.13.0-rc1
Optuna 0.7.0
OptKeras 0.0.1


## Set up Dataset

In [5]:
dataset_name = 'MNIST'

if dataset_name in ['MNIST', 'MNIST_1000samples']:
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    
    img_x, img_y = x_train.shape[1], x_train.shape[2]
    x_train = x_train.reshape(-1, img_x, img_y, 1)
    x_test = x_test.reshape(-1, img_x, img_y, 1)   
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    num_classes = 10
    input_shape = (img_x, img_y, 1)

11493376/11490434 [==============================] - 0s 0us/step


In [6]:
print('x_train: ', x_train.shape)
print('y_train', y_train.shape)
print('x_test: ', x_test.shape)
print('y_test', y_test.shape)
print('input_shape: ', input_shape )    

x_train:  (60000, 28, 28, 1)
y_train (60000,)
x_test:  (10000, 28, 28, 1)
y_test (10000,)
input_shape:  (28, 28, 1)


## A simple Keras model

In [7]:
model = Sequential()
model.add(Conv2D(
    filters = 32, 
    kernel_size = 3, 
    strides = 1,
    activation = 'relu', 
    input_shape = input_shape ))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
model.compile(optimizer = Adam(), 
            loss='sparse_categorical_crossentropy', metrics=['accuracy'])			  
model.fit(x_train, y_train, 
          validation_data = (x_test, y_test), shuffle = True,
          batch_size = 512, epochs = 2) 

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 16s 266us/step - loss: 0.4474 - acc: 0.8783 - val_loss: 0.2442 - val_acc: 0.9286
Epoch 2/2
60000/60000 [==============================] - 2s 33us/step - loss: 0.1953 - acc: 0.9448 - val_loss: 0.1480 - val_acc: 0.9614


## Optimization of a simple Keras model without pruning

In [8]:
study_name = dataset_name + '_Simple'

""" Step 1. Instantiate OptKeras class
You can specify arguments for Optuna's create_study method and other arguments 
for OptKeras such as enable_pruning. 
"""

ok = OptKeras(study_name=study_name)


""" Step 2. Define objective function for Optuna """

def objective(trial):
    
    """ Clear the backend (TensorFlow). See:
    https://www.tensorflow.org/api_docs/python/tf/keras/backend/clear_session
    """
    K.clear_session() 
    
    """ Step 2.1. Define parameters to try using methods of optuna.trial such as 
    suggest_categorical. In this simple demo, try 2*2*2*2 = 16 parameter sets: 
    2 values specified in list for each of 4 parameters 
    (filters, kernel_size, strides, and activation for convolution).
    """    
    model = Sequential()
    model.add(Conv2D(
        filters = trial.suggest_categorical('filters', [32, 64]), 
        kernel_size = trial.suggest_categorical('kernel_size', [3, 5]), 
        strides = trial.suggest_categorical('strides', [1, 2]), 
        activation = trial.suggest_categorical('activation', ['relu', 'linear']), 
        input_shape = input_shape ))
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer = Adam(), 
                loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    """ Step 2.2. Specify callbacks(trial) and keras_verbose in fit 
    (or fit_generator) method of Keras model
    """
    model.fit(x_train, y_train, 
              validation_data = (x_test, y_test), shuffle = True,
              batch_size = 512, epochs = 2,
              callbacks = ok.callbacks(trial), 
              verbose = ok.keras_verbose )  
    
    """ Step 2.3. Return trial_best_value (or latest_value) """
    return ok.trial_best_value

""" Step 3. Run optimize. 
Set n_trials and/or timeout (in sec) for optimization by Optuna
"""
ok.optimize(objective, timeout = 60) # 1 minute for demo

[2019-02-24 09:32:25.462880] Ready for optimization. (message printed as verbose is set to 1+)
[2019-02-24 09:32:25.807950] Latest trial id: None, value: None (None) | Best trial id: None, value: None, parameters: None
[2019-02-24 09:32:28.177527] Latest trial id: 0, value: 0.06769999961853024 (TrialState.COMPLETE) | Best trial id: 0, value: 0.06769999961853024, parameters: {'filters': 32, 'kernel_size': 3, 'strides': 2, 'activation': 'relu'}
[2019-02-24 09:32:31.043376] Latest trial id: 1, value: 0.04040000076293948 (TrialState.COMPLETE) | Best trial id: 1, value: 0.04040000076293948, parameters: {'filters': 64, 'kernel_size': 5, 'strides': 2, 'activation': 'relu'}
[2019-02-24 09:32:33.679953] Latest trial id: 2, value: 0.0556999998092651 (TrialState.COMPLETE) | Best trial id: 1, value: 0.04040000076293948, parameters: {'filters': 64, 'kernel_size': 5, 'strides': 2, 'activation': 'relu'}
[2019-02-24 09:32:39.486085] Latest trial id: 3, value: 0.029299999523162867 (TrialState.COMPLETE)

## Randomized Grid Search of a simple Keras model

In [9]:
study_name = dataset_name + '_GridSearch'

""" To run randomized grid search, set random_grid_search_mode True """
ok = OptKeras(study_name=study_name, random_grid_search_mode=True)

def objective(trial):
   
    K.clear_session()
    
    model = Sequential()
    model.add(Conv2D(
        filters = trial.suggest_categorical('filters', [32, 64]), 
        kernel_size = trial.suggest_categorical('kernel_size', [3, 5]), 
        strides = trial.suggest_categorical('strides', [1, 2]), 
        activation = trial.suggest_categorical('activation', ['relu', 'linear']), 
        input_shape = input_shape ))
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer = Adam(), 
                loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    model.fit(x_train, y_train, 
              validation_data = (x_test, y_test), shuffle = True,
              batch_size = 512, epochs = 2,
              callbacks = ok.callbacks(trial), 
              verbose = ok.keras_verbose )  
    
    return ok.trial_best_value

""" Set the number of parameter sets as n_trials for complete grid search """
ok.random_grid_search(objective, n_trials = 2*2*2*2) # 2*2*2*2 = 16 param sets

[2019-02-24 09:33:25.975148] Ready for optimization. (message printed as verbose is set to 1+)
[2019-02-24 09:33:26.086711] Latest trial id: None, value: None (None) | Best trial id: None, value: None, parameters: None
[2019-02-24 09:33:31.771017] Completed:   6% (    1 /    16)
[2019-02-24 09:33:31.873750] Latest trial id: 0, value: 0.028599999523162833 (TrialState.COMPLETE) | Best trial id: 0, value: 0.028599999523162833, parameters: {'filters': 64, 'kernel_size': 3, 'strides': 1, 'activation': 'relu'}
[2019-02-24 09:33:34.324624] Completed:  12% (    2 /    16)
[2019-02-24 09:33:34.427215] Latest trial id: 1, value: 0.03540000038146973 (TrialState.COMPLETE) | Best trial id: 0, value: 0.028599999523162833, parameters: {'filters': 64, 'kernel_size': 3, 'strides': 1, 'activation': 'relu'}
[2019-02-24 09:33:39.776762] Completed:  19% (    3 /    16)
[2019-02-24 09:33:39.880299] Latest trial id: 2, value: 0.08099999942779545 (TrialState.COMPLETE) | Best trial id: 0, value: 0.028599999523

## Optimization of a Keras model using more Optuna's features such as pruning

In [10]:
study_name = dataset_name + '_Optimized'

ok = OptKeras( 
    # parameters for optuna.create_study
    storage='sqlite:///' + study_name + '_Optuna.db', 
    sampler=optuna.samplers.TPESampler(
        consider_prior=True, prior_weight=1.0, 
        consider_magic_clip=True, consider_endpoints=False, 
        n_startup_trials=10, n_ei_candidates=24, 
        seed=None), 
    pruner=optuna.pruners.SuccessiveHalvingPruner(
        min_resource=1, reduction_factor=4, min_early_stopping_rate=0), 
    study_name = study_name, 
    direction='minimize', 
    load_if_exists = True,
    # parameters for OptKeras
    monitor='val_error', # Either 'val_error' (1 - val_acc) or 'val_loss'
    enable_pruning=True, 
    models_to_keep=1, # Either 1, 0, or -1 (save all models) 
    verbose=1 )

def objective(trial): 
    epochs = 10
    
    K.clear_session()   
    model = Sequential()
    
    if trial.suggest_int('Conv', 0, 1):  
        # 1 Convolution layer
        i = 1
        model.add(Conv2D(
            filters = int(trial.suggest_discrete_uniform(
                'Conv_{}_num_filters'.format(i), 32, 64, 32)), 
            kernel_size=tuple([trial.suggest_int(
                'Conv_{}_kernel_size'.format(i), 2, 3)] * 2),
            activation='relu',
            input_shape = input_shape))
        model.add(MaxPooling2D(pool_size=tuple([trial.suggest_int(
                'Conv_{}_max_pooling_size'.format(i), 2, 3)] * 2)))
        model.add(Dropout(trial.suggest_discrete_uniform(
                'Conv_{}_dropout_rate'.format(i), 0, 0.5, 0.25) ))
        model.add(Flatten())        
    else:
        model.add(Flatten(input_shape=input_shape))
    # 2 Fully connected layers
    for i in np.arange(2) + 1:
        model.add(Dense(int(trial.suggest_discrete_uniform(
            'FC_{}_num_hidden_units'.format(i), 256, 512, 256))))
        if trial.suggest_int('FC_{}_batch_normalization'.format(i), 0, 1):
            model.add(BatchNormalization())
        model.add(Activation(trial.suggest_categorical(
            'FC_{}_acivation'.format(i), ['relu'])))
        model.add(Dropout(
            trial.suggest_discrete_uniform(
                'FC_{}_dropout_rate'.format(i), 0, 0.5, 0.25) ))
        
    # Output layer    
    model.add(Dense(num_classes, activation='softmax'))
    
    optimizer_dict = { \
    #'Adagrad': Adagrad(),
    'Adam': Adam() }
    
    model.compile(optimizer = optimizer_dict[
        trial.suggest_categorical('Optimizer', list(optimizer_dict.keys()))],
          loss='sparse_categorical_crossentropy', metrics=['accuracy'])    
    
    if ok.verbose >= 2: model.summary()
    
    batch_size = trial.suggest_int('Batch_size', 256, 256) 
    #batch_size = int(trial.suggest_discrete_uniform(
    #                  'Batch_size', 256, 512, 256) )
    data_augmentation = trial.suggest_int('Data_augmentation', 0, 1)
    
    if not data_augmentation:
        # [Required] Specify callbacks(trial) in fit method
        model.fit(x_train, y_train, batch_size = batch_size,
                  epochs = epochs, validation_data = (x_test, y_test),
                  shuffle = True,
                  callbacks = ok.callbacks(trial), 
                  verbose = ok.keras_verbose )
    
    if data_augmentation:
        # This will do preprocessing and realtime data augmentation:
        datagen = ImageDataGenerator(
            width_shift_range=[-1, 0, +1], # 1 pixel
            height_shift_range=[-1, 0, +1], # 1 pixel
            zoom_range=[0.95,1.05],  # set range for random zoom
            horizontal_flip=False,  # disable horizontal flip
            vertical_flip=False )  # disable vertical flip
        datagen.fit(x_train)
        # [Required] Specify callbacks(trial) in fit_generator method
        model.fit_generator(datagen.flow(x_train, y_train, 
                                         batch_size=batch_size),
                            epochs=epochs, validation_data=(x_test, y_test),
                            steps_per_epoch=len(x_train) // batch_size,
                            callbacks = ok.callbacks(trial), 
                            verbose = ok.keras_verbose )  
    
    # [Required] return trial_best_value (recommended) or latest_value
    return ok.trial_best_value

# Set n_trials and/or timeout (in sec) for optimization by Optuna
ok.optimize(objective, timeout = 10*60) # 10 minutes for demo

[2019-02-24 09:35:14.678466] Ready for optimization. (message printed as verbose is set to 1+)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
[2019-02-24 09:35:15.537035] Latest trial id: None, value: None (None) | Best trial id: None, value: None, parameters: None
[2019-02-24 09:35:52.461026] Latest trial id: 1, value: 0.012700000000000045 (TrialState.COMPLETE) | Best trial id: 1, value: 0.012700000000000045, parameters: {'Batch_size': 256, 'Conv': 1, 'Conv_1_dropout_rate': 0.5, 'Conv_1_kernel_size': 2, 'Conv_1_max_pooling_size': 2, 'Conv_1_num_filters': 32.0, 'Data_augmentation': 0, 'FC_1_acivation': 'relu', 'FC_1_batch_normalization': 0, 'FC_1_dropout_rate': 0.0, 'FC_1_num_hidden_units': 256.0, 'FC_2_acivation': 'relu', 'FC_2_batch_normalization': 1, 'FC_2_dropout_rate': 0.0, 'FC_2_num_hidden_units': 256.0, 'Optimizer': 'Adam'}
[2019-02-24 09:37:58.406814] Latest trial id: 5, value: 0.010399999999999965 (TrialState.

## Check the results

In [11]:
# OptKeras best_trial returns 
print('Best trial id: ', ok.best_trial.trial_id)
print('Best value:', ok.best_trial.value)
print('Best params: ')
ok.best_trial.params

Best trial id:  26
Best value: 0.008399999999999963
Best params: 


{'Batch_size': 256,
 'Conv': 1,
 'Conv_1_dropout_rate': 0.5,
 'Conv_1_kernel_size': 3,
 'Conv_1_max_pooling_size': 2,
 'Conv_1_num_filters': 64.0,
 'Data_augmentation': 1,
 'FC_1_acivation': 'relu',
 'FC_1_batch_normalization': 0,
 'FC_1_dropout_rate': 0.25,
 'FC_1_num_hidden_units': 512.0,
 'FC_2_acivation': 'relu',
 'FC_2_batch_normalization': 1,
 'FC_2_dropout_rate': 0.25,
 'FC_2_num_hidden_units': 256.0,
 'Optimizer': 'Adam'}

In [12]:
"""
Alternatively, you can access Optuna's study object to, for example, 
get the best parameters as well.
Please note that study.best_trial returns error if optimization trials 
were not completed (e.g. if you interupt execution) as of Optuna 0.7.0, 
so usage of OptKeras is recommended.
"""
study = ok.study
study.best_trial.params 

{'Batch_size': 256,
 'Conv': 1,
 'Conv_1_dropout_rate': 0.5,
 'Conv_1_kernel_size': 3,
 'Conv_1_max_pooling_size': 2,
 'Conv_1_num_filters': 64.0,
 'Data_augmentation': 1,
 'FC_1_acivation': 'relu',
 'FC_1_batch_normalization': 0,
 'FC_1_dropout_rate': 0.25,
 'FC_1_num_hidden_units': 512.0,
 'FC_2_acivation': 'relu',
 'FC_2_batch_normalization': 1,
 'FC_2_dropout_rate': 0.25,
 'FC_2_num_hidden_units': 256.0,
 'Optimizer': 'Adam'}

In [13]:
## Check the Optuna CSV log file 
pd.options.display.max_rows = 8 # limit rows to display
print('Data Frame read from', ok.optuna_log_file_path, '\n')
pd.read_csv(ok.optuna_log_file_path)

Data Frame read from MNIST_Optimized_Optuna.csv 



,trial_id,state,value,datetime_start,datetime_complete,Batch_size,Conv,Conv_1_dropout_rate,Conv_1_kernel_size,Conv_1_max_pooling_size,...,intermediate_values_0,intermediate_values_1,intermediate_values_2,intermediate_values_3,intermediate_values_4,intermediate_values_5,intermediate_values_6,intermediate_values_7,intermediate_values_8,intermediate_values_9
0,1,TrialState.COMPLETE,0.0127,2019-02-24 09:35:14.695532,2019-02-24 09:35:51.514608,256,1,0.5,2.0,2.0,...,0.0262,0.0171,0.0151,0.0142,0.0149,0.0127,0.0127,0.0112,0.0136,0.0127
1,2,TrialState.PRUNED,0.0280,2019-02-24 09:35:51.549683,2019-02-24 09:36:28.213672,256,0,NaN,NaN,NaN,...,0.0357,0.0280,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,TrialState.PRUNED,0.0307,2019-02-24 09:36:28.232243,2019-02-24 09:36:34.286224,256,0,NaN,NaN,NaN,...,0.0371,0.0307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,TrialState.PRUNED,0.0257,2019-02-24 09:36:34.306324,2019-02-24 09:37:11.722880,256,0,NaN,NaN,NaN,...,0.0451,0.0257,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22,23,TrialState.PRUNED,0.0370,2019-02-24 09:44:19.715208,2019-02-24 09:44:25.254138,256,0,NaN,NaN,NaN,...,0.0507,0.0370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,24,TrialState.PRUNED,0.0315,2019-02-24 09:44:25.289811,2019-02-24 09:45:00.688242,256,0,NaN,NaN,NaN,...,0.0440,0.0315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,25,TrialState.PRUNED,0.0179,2019-02-24 09:45:00.721059,2019-02-24 09:45:12.520694,256,1,0.5,2.0,2.0,...,0.0270,0.0179,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,26,TrialState.COMPLETE,0.0084,2019-02-24 09:45:12.556062,2019-02-24 09:48:22.300769,256,1,0.5,3.0,2.0,...,0.0158,0.0130,0.0098,0.0083,0.0084,0.0096,0.0095,0.0083,0.0081,0.0084


In [14]:
## Check the Keras CSV log file

pd.options.display.max_rows = 8 # limit rows to display
print('Data Frame read from', ok.keras_log_file_path, '\n')
pd.read_csv(ok.keras_log_file_path)

Data Frame read from MNIST_Optimized_Keras.csv 



,epoch,_Datetime_epoch_begin,_Datetime_epoch_end,_Monitor,_Trial_id,acc,error,loss,val_acc,val_error,val_loss
0,0,2019-02-24 09:35:16.186699,2019-02-24 09:35:19.896706,val_error,1,0.934550,0.065450,0.215014,0.9738,0.0262,0.087519
1,1,2019-02-24 09:35:20.358237,2019-02-24 09:35:23.464105,val_error,1,0.976250,0.023750,0.077705,0.9829,0.0171,0.053412
2,2,2019-02-24 09:35:23.823867,2019-02-24 09:35:26.924000,val_error,1,0.982567,0.017433,0.056076,0.9849,0.0151,0.048943
3,3,2019-02-24 09:35:27.455028,2019-02-24 09:35:30.547219,val_error,1,0.986300,0.013700,0.041604,0.9858,0.0142,0.042347
...,...,...,...,...,...,...,...,...,...,...,...
86,6,2019-02-24 09:47:08.107669,2019-02-24 09:47:25.428471,val_error,26,0.984400,0.015600,0.048308,0.9905,0.0095,0.029372
87,7,2019-02-24 09:47:26.180463,2019-02-24 09:47:44.109124,val_error,26,0.986509,0.013491,0.043048,0.9917,0.0083,0.026503
88,8,2019-02-24 09:47:44.862322,2019-02-24 09:48:02.128197,val_error,26,0.986777,0.013223,0.041968,0.9919,0.0081,0.026766
89,9,2019-02-24 09:48:03.942366,2019-02-24 09:48:21.514279,val_error,26,0.987597,0.012403,0.038953,0.9916,0.0084,0.025915
